In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings("ignore")

2025-09-27 01:20:14.895489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758936015.085594      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758936015.140079      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/emnist-jpeg/EMNIST/emnist.csv")

In [3]:
for ind in range(0,len(df["img_name"])):
    df["img_name"][ind] = "/kaggle/input/emnist-jpeg/EMNIST/" + df["img_name"][ind]

In [4]:
df["img_name"][112799]

'/kaggle/input/emnist-jpeg/EMNIST/8/char_112799.jpeg'

In [5]:
df

,img_name,label
0,/kaggle/input/emnist-jpeg/EMNIST/45/char_0.jpeg,45
1,/kaggle/input/emnist-jpeg/EMNIST/36/char_1.jpeg,36
2,/kaggle/input/emnist-jpeg/EMNIST/43/char_2.jpeg,43
3,/kaggle/input/emnist-jpeg/EMNIST/15/char_3.jpeg,15
4,/kaggle/input/emnist-jpeg/EMNIST/4/char_4.jpeg,4
...,...,...
112795,/kaggle/input/emnist-jpeg/EMNIST/22/char_11279...,22
112796,/kaggle/input/emnist-jpeg/EMNIST/17/char_11279...,17
112797,/kaggle/input/emnist-jpeg/EMNIST/23/char_11279...,23
112798,/kaggle/input/emnist-jpeg/EMNIST/31/char_11279...,31


In [6]:
x = df["img_name"]
y = df["label"].astype("str")
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,random_state=42)
train = pd.DataFrame({"img_name":x_train,"label":y_train})
test = pd.DataFrame({"img_name":x_test,"label":y_test})

In [7]:
generator = ImageDataGenerator()
test_generator = ImageDataGenerator()

In [8]:
train_gen = generator.flow_from_dataframe(
    dataframe=train,
    x_col = 'img_name',
    y_col = "label",
    target_size = (128,128),
    batch_size = 64,
    class_mode='sparse',
)
test_gen = test_generator.flow_from_dataframe(
    dataframe=test,
    x_col = 'img_name',
    y_col = "label",
    target_size = (128,128),
    batch_size = 64,
    class_mode='sparse',
)


Found 101520 validated image filenames belonging to 47 classes.
Found 11280 validated image filenames belonging to 47 classes.


In [9]:
num_classes = len(list(set(df["label"])))
num_classes

47

In [10]:
model = Sequential([Conv2D(16,(3,3),activation='relu',input_shape=(128,128,3),kernel_regularizer = l2(0.03)),
                    Dropout(0.25),
                   Conv2D(32,(3,3),activation='relu',kernel_regularizer = l2(0.03),padding="same"),
                    MaxPooling2D((2,2)),
                   Dropout(0.25),
                    BatchNormalization(),
                    Conv2D(64,(3,3),activation='relu',kernel_regularizer = l2(0.03),padding="same"),
                   Dropout(0.25),
                    Conv2D(128,(3,3),activation='relu',kernel_regularizer = l2(0.03)),
                   Dropout(0.25),
                    BatchNormalization(),
                    Conv2D(256,(3,3),activation='relu',kernel_regularizer = l2(0.03),padding="same"),
                   MaxPooling2D((2,2)),
                   Dropout(0.25),
                    Conv2D(512,(3,3),activation='relu',kernel_regularizer = l2(0.03),padding="same"),
                    MaxPooling2D((2,2)),
                   Dropout(0.25),
                    Flatten(),
                   Dense(1024,activation='relu',kernel_regularizer=l2(0.03)),
                    BatchNormalization(),
                    Dropout(0.45),
                   Dense(num_classes,activation='softmax')])

I0000 00:00:1758936378.768443      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758936378.769156      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 126, 126, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 126, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 63, 63, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 63, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 61, 61, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 30, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │   117,965,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 47)             │        48,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,591,503 (456.21 MB)

 Trainable params: 119,589,135 (456.20 MB)

 Non-trainable params: 2,368 (9.25 KB)

In [12]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics = ["accuracy"])

In [13]:
lr_sch = ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3,verbose=1)

In [14]:
run = model.fit(train_gen,validation_data=test_gen,epochs=5,callbacks=[lr_sch],verbose=1)

Epoch 1/5


I0000 00:00:1758936387.273750      62 service.cc:148] XLA service 0x78cdc0003c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758936387.274564      62 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1758936387.274588      62 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1758936387.923472      62 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1758936409.199117      62 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1587/1587 ━━━━━━━━━━━━━━━━━━━━ 506s 301ms/step - accuracy: 0.5378 - loss: 11.2569 - val_accuracy: 0.4403 - val_loss: 2.6526 - learning_rate: 0.0010
Epoch 2/5
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 350s 220ms/step - accuracy: 0.7009 - loss: 1.6667 - val_accuracy: 0.7204 - val_loss: 1.5379 - learning_rate: 0.0010
Epoch 3/5
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 345s 217ms/step - accuracy: 0.7217 - loss: 1.5247 - val_accuracy: 0.6978 - val_loss: 1.5668 - learning_rate: 0.0010
Epoch 4/5
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 345s 217ms/step - accuracy: 0.7262 - loss: 1.4907 - val_accuracy: 0.7274 - val_loss: 1.4682 - learning_rate: 0.0010
Epoch 5/5
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 345s 217ms/step - accuracy: 0.7282 - loss: 1.4884 - val_accuracy: 0.7331 - val_loss: 1.4203 - learning_rate: 0.0010
